In [1]:
import pandas as pd
import numpy as np
import scipy
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import re
from scipy.linalg import norm
from scipy import stats
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

In [2]:
# Загружаем стоп слова для русского и английского языка
stop1 = list(stopwords.words('english'))
stop2 = list(stopwords.words('russian'))

stop = stop1 + stop2



lemmatizer = WordNetLemmatizer()

# Функция для предобработки. Удаление знаков пуктуации, приведение к нижнему регистру, лемматизация.
def preprocessing(line):
    line = line.lower()
    line = re.sub(r'[.,"\'-?:!;]', "", line)
    line = nltk.word_tokenize(line)
    line = ' '.join([lemmatizer.lemmatize(w) for w in line])
    return line


In [3]:
# Читаем наш датасет.
df = pd.read_csv('train.csv')

In [4]:
# Посмотрим на данные.
df.head()

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497819 entries, 0 to 497818
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   pair_id       497819 non-null  int64 
 1   name_1        497819 non-null  object
 2   name_2        497819 non-null  object
 3   is_duplicate  497819 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 15.2+ MB


In [6]:
# Посмотрим сколько одинаковых пар, а сколько разных.
df.is_duplicate.value_counts()

0    494161
1      3658
Name: is_duplicate, dtype: int64

In [7]:
# Разбиваем датафрейм на на части с копиями и без них.

df_0 = df[df['is_duplicate'] == 0]
df_1 = df[df['is_duplicate'] == 1]

name_1_df_0 = list(df_0['name_1'])
name_2_df_0 = list(df_0['name_2'])

name_1_df_1 = list(df_1['name_1'])
name_2_df_1 = list(df_1['name_2'])

In [8]:
# Объединим два столбца, посмотрим сколько всего названий.
text = list(df['name_1']) + list(list(df['name_2']))

print(len(text))

995638


In [9]:
# Оставим только уникальные названия.

text_unic = list(set(text))

print(len(text_unic))

# Разобъем все названия на отдельные слова
words = []
for string in text_unic:
    subwords = string.split()
    for word in subwords:
        words.append(word)

print("Количество слов" , len(words))
words_unic = list(set(words))
print("Количество уникальных слов ", len(words_unic))

18022
Количество слов 74073
Количество уникальных слов  18177


In [10]:
# Посмотрим на результат предобработки
print(preprocessing(text[12]))

bestocean worldwide logistics inc


In [11]:
# Будем делать эмбеддинги с помощью TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(analyzer = 'word', stop_words=stop)

tfidf_vectorizer.fit(words_unic)

TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [12]:
# Пример текста. Предобработаем его, вычислим эмбеддинги, а потом посмотрим на косинусное сходство.

text1 = preprocessing('Carlisle Coatings & Waterproofing, Inc.' )
text2 = preprocessing('Carlisle Coatings & Wtrprfng')

print(text1)
print(text2)

carlisle coating & waterproofing inc
carlisle coating & wtrprfng


In [13]:
# Функция делает эмбеддинг названия, как сумму эмбеддингов входящих в него слов.
def sentence_embedding(string):
    sentence_list = [tfidf_vectorizer.transform([word]) for word in string.split()]
    result = np.sum(np.array(sentence_list), axis=0)
    return result

    
text1_vec = sentence_embedding(text1)
text2_vec = sentence_embedding(text2)

In [14]:
# Посмотрим на косинусное сходство двух текстов разных и одинаковых.

print(cosine_similarity(text1_vec, text2_vec))
print(cosine_similarity(text1_vec, text1_vec))

[[0.57735027]]
[[1.]]


In [16]:
# Вычислим косинусное сходство для всех пар из нашего датасета.

pair_0 = [cosine_similarity( sentence_embedding(item[0] ), sentence_embedding(item[1]) ) for item in zip(name_1_df_0 ,name_2_df_0 )]
pair_1 = [cosine_similarity( sentence_embedding(item[0]), sentence_embedding(item[1]) ) for item in zip(name_1_df_1 ,name_2_df_1 )]    

In [17]:
# Посмотрим на косинусное сходство первых 50 непохожиш пар.
print(pair_0[:50])

[array([[0.25819889]]), array([[0.57735027]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.5976143]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.28867513]]), array([[0.25]]), array([[0.25819889]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.31622777]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.21821789]]), array([[0.28867513]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.33333333]]), array([[0.]]), array([[0.23570226]]), array([[0.67082039]]), array([[0.42426407]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.26726124]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.3012992]]), array([[0.]]), array([[0.6]]), array([[0.]]), array([[0.]]), array([[0.]])]


In [18]:
# Посмотрим на косинусное сходство первых 50 похожиш пар.
print(pair_1[:50])

[array([[0.6681531]]), array([[0.35355339]]), array([[0.35355339]]), array([[0.43643578]]), array([[0.25819889]]), array([[0.2236068]]), array([[0.5]]), array([[0.57735027]]), array([[0.25]]), array([[0.57735027]]), array([[0.66666667]]), array([[0.33333333]]), array([[0.40824829]]), array([[0.51639778]]), array([[0.51639778]]), array([[0.71428571]]), array([[1.]]), array([[0.25]]), array([[0.28867513]]), array([[0.81649658]]), array([[0.40824829]]), array([[0.35355339]]), array([[0.6]]), array([[0.33333333]]), array([[0.28867513]]), array([[0.66666667]]), array([[0.]]), array([[0.75]]), array([[1.]]), array([[0.47140452]]), array([[0.35355339]]), array([[0.35355339]]), array([[0.6]]), array([[0.5]]), array([[0.]]), array([[0.35355339]]), array([[0.28867513]]), array([[0.70710678]]), array([[0.70710678]]), array([[0.89442719]]), array([[0.70710678]]), array([[0.15430335]]), array([[0.25]]), array([[0.33333333]]), array([[0.35355339]]), array([[0.8660254]]), array([[0.2236068]]), array(

In [19]:
print(len(pair_0))
print(len(pair_1))

494161
3658


In [20]:
# Выведем долю пар схожих и отличных по косинусному сходству по порогу 0.3

pair_0_trashold_05 = [i for i in pair_0 if i < 0.3]
pair_1_trashold_05 = [i for i in pair_1 if i>= 0.3]

print("Количество непохожих пар ",len(pair_0_trashold_05), "  Доля", len(pair_0_trashold_05)/len(pair_0)*100," %")

print("Количество похожих пар ", len(pair_1_trashold_05), "  Доля", len(pair_1_trashold_05)/len(pair_1)*100, " %")

Количество непохожих пар  373625   Доля 75.60794963584743  %
Количество похожих пар  2589   Доля 70.77638053581192  %
